In [1]:
import numpy as np


def _set_up_pauli():
    """Creates Pauli matrices and identity
    """
    sigx = np.zeros((2, 2))
    sigz = np.zeros((2, 2))
    id2 = np.identity(2)
    proj0 = np.zeros((2, 2))
    proj1 = np.zeros((2, 2))
    sigplus = np.zeros((2, 2))
    sigminus = np.zeros((2, 2))
    sigx[0, 1] = 1.0
    sigx[1, 0] = 1.0
    sigz[0, 0] = 1.0
    sigz[1, 1] = -1.0
    proj0[0, 0] = 1.0
    proj1[1, 1] = 1.0
    sigplus[1, 0] = 1.0
    sigminus[0, 1] = 1.0
    return id2, sigx, sigz, proj0, proj1, sigplus, sigminus


ID2, SIG_X, SIG_Z, PROJ_0, PROJ_1, SIG_PLUS, SIG_MINUS = _set_up_pauli()


In [2]:
nqb=2



def pushtoFock(i, local):
    """Tensor product of `local` at particle index i with 1 in fock space

    Arguments:
        i: particle index of matrix local
        local: matrix operator
    """
    fock = np.identity(1)
    for j in range(nqb):
        if j == i:
            fock = np.kron(fock, local)
        else:
            fock = np.kron(fock, ID2)
    return fock

FockZ=[pushtoFock(i,SIG_Z) for i in range(nqb)]
FockZZ=[[FockZ[i]@FockZ[j] for i in range(nqb)] for j in range(nqb)]


def _constructIsingH(Jij, hi):
    """Computes Hamiltonian (``J_ij is i < j``, i.e., upper diagonal"""
    IsingH = np.zeros((2 ** nqb, 2 ** nqb))
    for i in range(nqb):
        IsingH += hi[i] * FockZ[i]
        for j in range(i):
            IsingH += Jij[j, i] * FockZZ[i][j]
    return IsingH

In [3]:
# no offset
Jij=np.array([[0.0,2.0],[2.0,0.0]])
hi=np.array([1.0,1.0])
IsingH=_constructIsingH(Jij, hi)
print(IsingH)

[[ 4.  0.  0.  0.]
 [ 0. -2.  0.  0.]
 [ 0.  0. -2.  0.]
 [ 0.  0.  0.  0.]]


In [5]:
# with offset
B=np.array([0.9,1.1])

for i in range(nqb):
    for j in range(nqb):
        Jij[i,j]=Jij[i,j]*np.sqrt(B[i] * B[j])
    hi[i]=hi[i]*B[i] 

IsingH=_constructIsingH(Jij, hi)
print(IsingH)

[[ 3.98997487  0.          0.          0.        ]
 [ 0.         -2.18997487  0.          0.        ]
 [ 0.          0.         -1.78997487  0.        ]
 [ 0.          0.          0.         -0.01002513]]
